# 분석 정리

1. 피처 변환 : Weekly_Sales -> log
2. 피처 추가 : Date -> year, month, week, day (주말은 어차피 금요일 밖에 없음)
3. 피처 제거 : Date -> 세분화했으니 제거
4. 피처 제거 : Fuel_Price -> Weekly_Sales와 상관관계가 보이지 않음, 그러나 파생 피처를 만들어서 쓸 수도 있음.
5. 피처 인코딩 : IsHoliday -> bool을 int로 숫자화 하기
6. 피처 결측치 : Promotion 1 ~ 5 -> 프로모션이 없는 것으로 간주해서 0으로 채움.

# 베이스라인 모델 전체 프로세스

데이터 불러오기 -> 피처 엔지니어링 -> 평가지표 계산 함수 작성 -> 모델 훈련 -> 성능 검증 -> 제출

## 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data_path = 'data/'

train = pd.read_csv(data_path + 'train.csv', index_col='id') # 학습 데이터
test = pd.read_csv(data_path + 'test.csv', index_col='id') # 테스트 데이터
submission = pd.read_csv(data_path + 'sample_submission.csv') # 제출 샘플 데이터

train.shape, test.shape, submission.shape

((6255, 12), (180, 11), (180, 2))

## 피처 엔지니어링

In [2]:
# 분석6 : 피처 결측치
train = train.fillna(0)
test = test.fillna(0)

In [3]:
# 데이터 합치기
df = pd.concat([train, test])

df.shape

(6435, 12)

In [4]:
# 분석1 : 피처 변환
df['Weekly_Sales'] = np.log1p(df['Weekly_Sales'])

In [5]:
# 분석2 : 피처 추가
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['week'] = df['day'] // 7

In [6]:
# 분석3, 4 : 피처 제거
drop_features = ['Date', 'Fuel_Price']
df = df.drop(columns=drop_features)

In [7]:
# 분석5 : 피처 인코딩
df['IsHoliday'] = df['IsHoliday'].astype(int)

In [8]:
# 데이터셋 나누기
X = df.loc[df['Weekly_Sales'].notnull()]
y = X.pop('Weekly_Sales')

test = df[df['Weekly_Sales'].isnull()].drop(columns='Weekly_Sales')

X.shape, y.shape, test.shape

((6255, 13), (6255,), (180, 13))

In [9]:
from sklearn.model_selection import train_test_split

# 검증 데이터 생성
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((5004, 13), (1251, 13), (5004,), (1251,))

## 평가지표 계산 함수 작성

RMSLE

In [10]:
from sklearn.metrics import mean_squared_error

def rmsle(y_val, y_pred):
    return np.sqrt(mean_squared_error(y_val, y_pred))

## 모델 훈련

In [11]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

## 성능 검증

In [12]:
prediction = lr_model.predict(X_val)

In [13]:
print(f"선형 회귀의 RMSLE 값 : {rmsle(y_val, prediction):.4f}")

선형 회귀의 RMSLE 값 : 0.5405


## 제출

In [14]:
y_pred = lr_model.predict(test)

In [15]:
submission['Weekly_Sales'] = np.expm1(y_pred) # 지수 변환
submission.to_csv(data_path + 'submission_baseline.csv', index=False)